# Project 1 Notebook for Jonathan Owens
This notebook is to keep track of the various items worked on with a place to collaborate all thoughts, ideas and creations with the team.


## Regional weather site code and elements for the region growing corn, wheat and soybeans
http://http://builder.rcc-acis.org/

- Station ID: KOMA
    - Location: Omaha Eppley Airfield
    - Region: Omaha, Nebrask
- Start date: 2000-10-26
- Elements: pcpn,avgt (percipitation, average temperature)
- End date: 2020-10-28
- Output: csv (comma seperated value)


## Regional Climate Centers (RCC) - Applied Climate Information Systems (ACIS) Documentation
    
- Documentation: http://www.rcc-acis.org/docs_webservices.html

- API URL: http://data.rcc-acis.org/StnData

- API URL ELEMENTS:

    elements = {
        sid=KOMA,
        sdate=20001026,
        elems=pcpn,avgt,
        edate=20201028,
        output=csv
        }

- Missing data values are returned as "M" and traces of precipitation, snowfall or snow depth are returned as "T".
